In [2]:
#Named Entity Recognition solution (proof of concept)

#Step 1: Importing files, locations and legal stop words
import pandas as pd
location=pd.read_csv ('/content/locations.tsv', header=None, sep='\t',names=['location','null'])
legal=pd.read_csv ('/content/legal.txt',header=None,names=["legal"])
company=pd.read_csv ('/content/companies.txt',header=None,  sep='\n',names=["names"])

#Step 2: All characters are lower case
location['location'] = location['location'].str.lower()
legal['legal'] = legal['legal'].str.lower()
company.names=company.names.str.lower()

#Step 3: Copies made (I know this is redundant, but I like to make copies)
companyback=company.copy()
location2=location['location']

#Step 4: Setting legal stop words
df=legal.values.flatten()

#Step 5: Setting location stop words
foo3=location.values.flatten()
location2 = location2.str.replace(' ', '_')
foo4=location2.values.flatten()
LEGAL_STOP_WORDS = set(df)
LOCATION_STOP_WORDS = set(foo3)

#Step 6: Company name can be considered as the raw corpus, without location and legal extention where company = corpus - (loc + legal)
import re
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^a-z #+_]')
def clean_compan_name(text):
    text = ' '.join(word for word in text.split() if word not in LOCATION_STOP_WORDS)
    text = ' '.join(word for word in text.split() if word not in LEGAL_STOP_WORDS) 
    return text
NAME = company['names'].apply(clean_compan_name)

#NB: It is possible to also use spacy, however I decided to do a bit of hand made NER
'''import spacy 
from spacy import displacy
nlp = spacy.load("en_core_web_sm")
sentence = str(NAME)
sentence_nlp = nlp(sentence)

# print named entities in article
print([(word, word.ent_type_) for word in sentence_nlp if word.ent_type_])

# visualize named entities
displacy.render(sentence_nlp, style='ent', jupyter=True)'''

#Step 7: Cleaning legal extensions 
def clean_legal(text3):
    text3 = ' '.join(word for word in text3.split() if word  in LEGAL_STOP_WORDS)
    return text3
LEGAL = company['names'].apply(clean_legal)
LEGAL = LEGAL.str.replace('\d+', '')

#Step 8: Defining locations (less succesfull)
leg=LEGAL.values.flatten()
LEGAL_STOP_WORDS2 = set(leg)

#Step 9: Deleting some common stop words found inside the datasets using .value_counts()
STOP_WORDS = set([
    'and',
    'of',
    'the',
    'co',
    'de',
    'ab',
    'la',
    'for',
    'in',
    'center',
    'law',
    'art',
    'north',
    'und',
    'as',
    'university',
    'mobile',
    'city',
    'spa',
    'one',
    'central',
    'market',
    'enterprise',
    'at',
    'i',
    'alliance',
    'van',
    'san',
    'aps',
    'human',
    'all',
   'cloud',
   'tmm ',
   'bay',
   'cah',
   'rat',
   'dog dog',
   'lal',
   'twin god',
   'rjn',
   'tag pet',
   'gil',
   'robert ace',
   'cos',
   'st',
   'union',
   'golden',
   'del',
   'des',
   'abc',
   'god'

   ])


#Step 10: Defining locations (less succesfull) location = corpus - (legal + assigned stop words - (corpus - legal) ) + location 
leg=LEGAL.values.flatten()
LEGAL_STOP_WORDS2 = set(leg) #Remove wrong labelled locations
def to_clean_loc(text4):
    text4 = REPLACE_BY_SPACE_RE.sub(' ', text4)
    text4 = BAD_SYMBOLS_RE.sub('', text4) 
    text4 = text4.replace('x', '') 
    text4 = ' '.join(word for word in text4.split() if word not in LEGAL_STOP_WORDS2)
    return text4

jnk = company['names'].apply(to_clean_loc)
jnk = jnk.str.replace('\d+', '')
junkloc=jnk.values.flatten()
JUNKLOC_STOP_WORDS = set(junkloc)

def clean_loc(text5):
    text5 = ' '.join(word for word in text5.split() if word not in LEGAL_STOP_WORDS2)
    text5 = ' '.join(word for word in text5.split() if word in LOCATION_STOP_WORDS)
    text5 = ' '.join(word for word in text5.split() if word not in JUNKLOC_STOP_WORDS)
    text5 = ' '.join(word for word in text5.split() if word not in STOP_WORDS)
    return text5

LOCATION = company['names'].apply(clean_loc)
LOCATION = LOCATION.str.replace('\d+', '')

#Step 11: Merging and saving
frames = [NAME, LOCATION, LEGAL]

result = pd.concat([NAME, LOCATION, LEGAL], axis=1)

result.to_csv('/content/ds.csv')


#Step 12: Quick Cleaning

df=pd.read_csv('/content/ds.csv', index_col=None)
df.rename(columns={'names.1': 'location', 'names.2': 'legal'}, inplace=True)
del df['Unnamed: 0']
df=df.fillna('unknown')

#Step 13: NameComponents
class NameComponents: 
    def get_name_components(self,comp_name): 
        self.comp_name= comp_name 
        name='chocolat frey'                   
        string = "{ 'raw':company.loc[company.iloc[:,0].str.contains(name)],'base_name': df.loc[df.iloc[:,0].str.contains(name)], 'location': df.loc[df.iloc[:,1].str.contains(name)], 'legal':df.loc[df.iloc[:,2].str.contains(name)]}" 
        Dict = eval(string) 
        print ("'raw':",Dict['raw'])
        print ("'base_name':",Dict['base_name'])
        print ("'location':",Dict['location'])
        print ("'legal':",Dict['legal'])
                  
comp_name = NameComponents() 
comp_name.get_name_components(comp_name) 

'raw':                       names
1588  chocolat frey usa ltd
'base_name':               names location legal
1588  chocolat frey      usa   ltd
'location': Empty DataFrame
Columns: [names, location, legal]
Index: []
'legal': Empty DataFrame
Columns: [names, location, legal]
Index: []


In [ ]:
#Part 2, find legal without reference, can be found splitting out all the words and count them.
#Having this infos allows more filtering during the NER process
#Legal should appear on the top of this list, because they are often mentioned 
food_list=[]
words = NAME.str.split()
for i in words:
  food_list.append(i)
from collections import Counter
words = Counter(c for clist in food_list for c in clist)

In [83]:
print(words.get('ltd.'))
print(words.get('srls'))
print(words.get('pvt'))
print(words.get('pvt.'))#A bit of regex should be implemented to remove dots etc

1112
6
277
273


In [79]:
words.most_common()

[('&', 3673),
 ('-', 3640),
 ('limited', 2988),
 ('inc.', 2148),
 ('group', 1961),
 ('services', 1899),
 ('solutions', 1498),
 ('consulting', 1311),
 ('ltd.', 1112),
 ('management', 1009),
 ('design', 932),
 ('media', 913),
 ('international', 905),
 ('marketing', 747),
 ('company', 716),
 ('business', 677),
 ('systems', 636),
 ('e', 542),
 ('technologies', 529),
 ('technology', 510),
 ('global', 503),
 ('service', 493),
 ('studio', 493),
 ('holdings', 491),
 ('associates', 483),
 ('a', 475),
 ('school', 470),
 ('health', 452),
 ('co.', 445),
 ('digital', 441),
 ('capital', 439),
 ('development', 387),
 ('new', 385),
 ('engineering', 383),
 ('enterprises', 378),
 ('partners', 376),
 ('software', 357),
 ('hotel', 345),
 ('industries', 344),
 ('training', 330),
 ('home', 327),
 ('club', 327),
 ('network', 323),
 ('creative', 319),
 ('photography', 317),
 ('foundation', 316),
 ('real', 316),
 ('communications', 314),
 ('energy', 312),
 ('web', 310),
 ('association', 308),
 ('y', 308),
 ('c